**IMPORT**

In [8]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import yfinance as yf

import matplotlib as mpl
import matplotlib.pyplot as plt

import keras.applications as kapp  
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM 
try:
    import tensorflow
except:
    import traceback
    traceback.print_exc()

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
     
%matplotlib inline
mpl.rcParams['figure.dpi'] = 200
# plt.style.use("ggplot")

**DATA INGESTION AND EXPLORATION**

In [9]:
# df_portfolio = pd.read_csv("portfolio_template.csv")
# df_portfolio.info()

In [16]:
def stock_dataframe(name: str = "stock/ticker symbol"):
    _stock_info = yf.Ticker(name)
    df = _stock_info.history(period="max")
    df = df.drop(['Dividends','Stock Splits'], axis=1)
    df = df[df.index>'1990-01-01'][df.index<'2020-03-01']
    df = df[df['Volume']>0]
    df['Year-Week'] = df.index.strftime('%d-%U-%Y')
    df = df.set_index('Year-Week')
    return df

def plot_close_price(df):
    df.plot.line(y="Close",use_index=True,)

def get_sub_df_combined(df,divs):
    no_of_plots = int(df.count()[0]/divs)
    fig, axs = plt.subplots(int(no_of_plots/2)+1, 2, figsize=(80, 80))
    fig.suptitle('Vertically stacked subplots')
    _count=1
    for i in range(no_of_plots):
        _df = df.iloc[i*divs:i*divs+divs]
        if _count==1:
            axs[i//2][0].plot(_df.index, _df['Close'])
            axs[i//2][0].set_xticklabels(_df.index,rotation=45, ha='right')
            _count+=1
        else:
            axs[i//2][1].plot(_df.index, _df['Close'])
            axs[i//2][1].set_xticklabels(_df.index,rotation=45, ha='right')
            _count = 1
    
    plt.savefig('foo.png', bbox_inches='tight')

def get_sub_df_separate(df,divs):
    no_of_plots = int(df.count()[0]/divs)
    script_dir = os.path.curdir
    results_dir = os.path.join(script_dir, 'Outputs/')

    if not os.path.isdir(results_dir):
        os.makedirs(results_dir)

    for i in range(no_of_plots):
        _df = df.iloc[i*divs:i*divs+divs]
        plt.figure(figsize = (40,20))
        plt.plot(_df.index, _df['Close'])
        plt.xticks(rotation=45)
        file_name = f"par{i}.png"
        plt.savefig(results_dir+file_name, bbox_inches='tight')
        plt.clf()
def get_one_plot(df,divs):
    no_of_plots = int(df.count()[0]/divs)
    script_dir = os.path.curdir
    results_dir = os.path.join(script_dir, 'Outputs/')

    if not os.path.isdir(results_dir):
        os.makedirs(results_dir)

    for i in range(no_of_plots):
        _df = df.iloc[i*divs:i*divs+divs]
        plt.figure(figsize = (40,20))
        plt.plot(_df.index, _df['Close'])
        plt.xticks(rotation=45)
        file_name = f"par{i}.png"
        plt.savefig(results_dir+file_name, bbox_inches='tight')

In [11]:
df = stock_dataframe("^NSEI")
# plot_close_price(df)

In [12]:
# get_sub_df_separate(df,90)
# get_sub_df_combined(df,90)

In [14]:

train_size = int(len(df) * 0.8)
train_set = df.iloc[:train_size,:]
test_set = df.iloc[train_size:,:]

# Scale the data using a MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
train_set_scaled = scaler.fit_transform(train_set)
test_set_scaled = scaler.transform(test_set)

# Create a function to create a dataset with X timesteps and y values
def create_dataset(dataset, lookback=1):
    X, y = [], []
    for i in range(lookback, len(dataset)):
        X.append(dataset[i-lookback:i, 0])
        y.append(dataset[i, 0])
    return np.array(X), np.array(y)

# Create the training and testing datasets
lookback = 60
X_train, y_train = create_dataset(train_set_scaled, lookback)
X_test, y_test = create_dataset(test_set_scaled, lookback)

# Reshape the datasets to be compatible with LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Define the LSTM model architecture
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=64)

# Make predictions on the testing dataset
y_pred = model.predict(X_test)

# Scale the predictions back to the original range
y_pred = scaler.inverse_transform(y_pred)

# Calculate the root mean squared error (RMSE) of the predictions
rmse = np.sqrt(np.mean((y_pred - y_test)**2))
print("Root Mean Squared Error: {:.2f}".format(rmse))

FileNotFoundError: [Errno 2] No such file or directory: 'dataframe.csv'